In [1]:
%pip install scipy
%pip install matplotlib
%pip install ipywidgets
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatLogSlider, IntSlider
from scipy.integrate import solve_ivp, cumulative_trapezoid
import numpy as np
from collections import namedtuple
import plots

mu_0 = 4 * np.pi * 1e-7  # Permeability of free space in T·m/A

# Projectile properties
proj_diameter = 0.005  # Diameter in meters
proj_length = 0.010  # Length in meters
proj_density = 7870  # Density of iron in kg/m^3
proj_volume = np.pi * (proj_diameter / 2) ** 2 * proj_length  # Volume in cubic meters
proj_mass = proj_density * proj_volume  # Mass in kg

SimulationParameters = namedtuple('SimulationParameters',
                                  ['init_volts',
                                   'capacitance',
                                   'resistance',
                                   'num_turns',
                                   'coil_diameter',
                                   'coil_length',
                                   'duration_s'])

# Define the differential equation for the RLC circuit
def sys_model(t, y, params):
    # Destructure the state vector
    [voltage, current, acceleration] = y

    # Rate of change in voltage
    dVdt = current / params.capacitance

    # Compute inductance, L
    coil_area = np.pi * (params.coil_diameter / 2) ** 2  # Coil cross-section, in m^2
    L = (params.num_turns ** 2 * mu_0 * coil_area) / params.coil_length  # Inductance in Henrys    

    # Rate of change in current 
    dIdt = -(params.resistance * current + voltage) / L

    # Compute force and acceleration on the projective
    mu_r = 5000  # Relative permeability of soft iron
    force = (1 / 2) * mu_0 ** 3 * mu_r * coil_area * params.num_turns ** 2 * current ** 2
    dvdt = force / proj_mass
    return [dVdt, dIdt, dvdt]

def run_sim(params):
    # Initial voltage across cap, init current across inductor, init velocity of projectile
    init_conditions = [-params.init_volts, 0, 0]
    t_span = (0, params.duration_s)  # Simulation time span
    t_eval = np.arange(0,params.duration_s, 1e-6)  # Set simulation time step
    # Solve the differential equation
    return solve_ivp(
        sys_model,
        t_span,
        init_conditions,
        t_eval=t_eval,
        args=(params,))

solution=None
simulationParameters=None

def solve_and_plot(V0, C, R, N, D, l, duration_s):
    global simulationParameters
    global solution
    simulationParameters = SimulationParameters(init_volts=V0,
                                     capacitance=C,
                                     resistance=R,
                                     num_turns=N,
                                     coil_diameter=D,
                                     coil_length=l,
                                     duration_s=duration_s)
    solution = run_sim(simulationParameters)
    plots.plot_circuit(solution)

plots.start(solve_and_plot)


interactive(children=(IntSlider(value=100, description='V0 (Volts)', max=1000, step=5), FloatLogSlider(value=0…